# Valor en Riesgo (VaR) and CVaR

**Definición de Valor en Riesgo (VaR)**

El VaR mide la pérdida esperada máxima en un horizonte temporal y con un nivel de confianza dado.

**Definición formal:**


$$ VaR_{\alpha}(L) = \inf( l: \mathbb{P} (L\leq l) \geq \alpha $$

donde:

$L$: pérdidas del portafolio

$\alpha$: nivel de confianza (ej. 95% o 99%)

**Interpretación:**

Un VaR al 99% a un (1) día del -2% significa que:

1. Con un 99% de (nivel) de confianza, la pérdida diaria no excederá 2%.
2. Hay una probabibilidad del 1% de obtener una pérdida mayor al 2%.

**Horizonte temporal:**

Usualmente se calcula 1 día y 10 días (según Basilea).

## Métodos de Cálculo del VaR

1. Método Paramétrico (Delta-VaR)

2. Método Histórico

3. Simulación de Monte Carlo


### Ejemplo Numérico en Python (Colab)

In [ ]:
# # Importar bibliotecas/librerias necesarias
import numpy as np, pandas as pd, matplotlib.pyplot as plt, seaborn as sns, yfinance as yf
from scipy.stats import norm, t  # Distribucion normal
import warnings  # Manejo de warnings en las salidas del codigo
warnings.filterwarnings('ignore')

In [ ]:
# Informacion de los activos
acciones = ["AAPL","HD","MCD", "MMM"]
start = "2020-09-01"     # Fecha inicial
end   = "2025-09-19"     # Fecha final
# Descargar precios de las acciones
precios = yf.download(acciones, start=start, end=end)["Close"]  # Periodo total de analisis, precios diarios
retornos = np.log(precios / precios.shift(1)).dropna()      # Para retornos dicsretos: data.pct_change().dropna()

Para un portafolio de inversión. Por ejemplo, podemos tomar un portafolio equiponderado (sin aplicar modelos de optimización).

In [ ]:
n = len(acciones)
w = np.repeat(1/n,n)     ## 2500 USD en cada acción: 10_000 USD en total -- ## (10_000 * w) / precios.iloc[0] ## No. de acciones
ret_port = retornos @ w  # Retornos historicos del portafolio

plt.figure(figsize=(3,3))
sns.histplot(ret_port,bins=20,kde=True);

In [ ]:
activo = retornos['AAPL']
plt.figure(figsize=(3,3))
sns.histplot(activo,bins=20,kde=True)
sns.histplot(ret_port,bins=20,kde=True);

In [ ]:
print('Riesgo activo: ', activo.std()*100, '%')
print('Riesgo portafolio: ', ret_port.std()*100, '%')

**Valor Histórico del portafolio**


In [ ]:
fechas = precios.index
t = len(ret_port)
valor_port = np.zeros(t+1)
valor_port[0] = 10_000
for i in range(1, t+1):
    valor_port[i] = valor_port[i-1] * np.exp(ret_port[i-1])

plt.figure(figsize=(10,3))
plt.plot(fechas,valor_port, label="Valor del portafolio en el periodo total")
plt.legend();

In [ ]:
print('Retorno total :', (valor_port[-1] / valor_port[0])-1)  # Crecmiento total
print('Retorno total :', np.exp(ret_port.sum())-1)            # Conversion retorno continuo

## 1. Método Paramétrico (Delta-Normal)

Asume que los retornos son normales.

Fórmula:

$$ VaR = 𝜇_p + z_𝛼 𝜎_p $$

donde:

$𝜇_p$: retorno esperado del portafolio

$𝜎_p$: desviación estándar del portafolio

$z_𝛼$: distribución normal estándar con percentil $(1−𝛼)$.

In [ ]:
# Cálculo del VaR al 99% de confianza

alpha = 0.99
inversion = 10_000

In [ ]:
# Resultados del calculo del VaR para los activos

mu = retornos.mean()
sigma = retornos.std()

# Percentil de la normal estándar
z_alpha = norm.ppf(1-alpha)

# VaR en porcentaje
var_pct = mu + z_alpha * sigma

# VaR en valor monetario
var_value = -var_pct * inversion

In [ ]:
activo = 'AAPL'
print("Resultado VaR Paramétrico (Delta-Normal) para un activo:")
print(f"Activo: Media= {mu[activo]:.4%}, Vol= {sigma[activo]:.4%},  VaR= {var_pct[activo]:.2%} → ${var_value[activo]:,.0f}")

**Calculos para el portafolio:**

In [ ]:
mu_port = ret_port.mean()
sigma_port = ret_port.std()
var_port_pct = mu_port + z_alpha * sigma_port
var_port_val = -var_port_pct * inversion

print("Resultado VaR Paramétrico (Delta-Normal) para el portafolio:")
print(f"Portafolio: Media={mu_port:.4%}, Vol={sigma_port:.4%}, VaR={var_port_pct:.2%} → ${var_port_val:,.0f}")

In [ ]:
# Visualización de distribución normal del portafolio
plt.figure(figsize=(4,3))
x = np.linspace(mu_port - 4*sigma_port, mu_port + 4*sigma_port, 100)
plt.plot(x, norm.pdf(x, mu_port, sigma_port), label="Distribución Normal")
plt.axvline(var_port_pct, color="red", linestyle="--", label=f"VaR 99%: {var_port_pct:.2%}")
plt.title("Distribución de Retornos Normal - Portafolio")
plt.xlabel("Retorno diario")
plt.ylabel("Densidad")
plt.legend()
plt.grid()
plt.show()

# <font color="red"> **Tarea** </font>
<font color="red"> Comparar el VaR histórico de todos los activos con el portafolio. Ya tenemos el de AAPL, faltán los demás!
</font>

**y ¿qué pasa si cambiamos el nivel de confianza en la estimación del VaR paramétrico?**

In [ ]:
# Percentil de la normal estandar
alphas = np.linspace(0.01, 0.05, 9)
var_pct_it = []
var_value_it = []

for alpha in alphas:
  z_alpha = norm.ppf(1-alpha)
  var_pct_i = mu_port + z_alpha * sigma_port
  var_pct_it.append(-var_pct_i)
  var_value_it.append(-var_pct_i * inversion)

plt.figure(figsize=(5,3))
plt.plot((1-alphas),var_pct_it, "o", label='VaR por cada nivel de confianza')
plt.plot((1-alphas),var_pct_it)
plt.xlabel("Nvel de confianza (95%-99%)")
plt.ylabel("Maxima perdida (en %)")
plt.legend()
plt.show();

In [ ]:
plt.figure(figsize=(5,3))
plt.plot((1-alphas),var_value_it, "o", label='VaR por cada nivel de confianza')
plt.plot((1-alphas),var_value_it)
plt.xlabel("Nvel de confianza (95%-99%)")
plt.ylabel("Maxima perdida (en USD)")
plt.legend()
plt.show();

## Método Histórico

Metodología no paramétrica que estima el riesgo de pérdida utilizando la distribución empírica de los retornos históricos. Se utilizan rendimientos históricos directamente.

**Procedimiento:**

* Recolectar los retornos históricos (Se utilizan datos de retornos pasados).

* Ordenar las pérdidas históricas (de menor a mayor

* Seleccionar el percentil $(1−𝛼)$.


In [ ]:
# Cuantil empírico
activo = 'AAPL'
alpha = 0.99
var_pct = np.percentile(retornos[activo], 100*(1-alpha))
# VaR en unidades monetarias
var_val = - var_pct * inversion

print("VaR Histórico del activo: ")
print(f"Activo:   {var_pct:.2%}   →  ${var_val:,.0f}")

In [ ]:
# Para el portafolio
var_port_pct = np.percentile(ret_port, 100*(1-alpha))      # VaR en porcentaje
var_port_val = - var_port_pct * inversion                  # VaR en unidades monetarias

print("VaR Histórico del portafolio: ")
print(f"Activo:  {var_port_pct:.2%}   →  ${var_port_val:,.0f}")

In [ ]:
plt.figure(figsize=(4,3))
plt.hist(ret_port, bins=30,density=True)
plt.axvline(x=-var_port_pct, color='red', linestyle='--',label=f'VaR 99%: {-var_port_pct:.2%}')
plt.title('Distribución de Rendimientos')
plt.xlabel('Rendimiento diario')
plt.ylabel('Densidad')
plt.legend()
plt.grid()

# <font color="red"> **Tarea** </font>
<font color="red"> Comparar el VaR histórico de todos los activos con el portafolio. Ya tenemos el de AAPL, faltán los demás!
</font>

## VaR con simulación de Monte Carlo (MC)

Monte Carlo es una herramienta extremadamente eficaz para la aproximación numérica, ya que se trata de un método matemático computarizado que se utiliza para producir una estimación cuando no existe una solución de forma cerrada. Las muestras aleatorias repetidas de una distribución determinada son la base del análisis de Monte Carlo.

Genera escenarios aleatorios según una distribución supuesta. Permite modelar colas más pesadas.

El método matemático Monte Carlo se puede definir de la siguiente manera:

Sea ($x_1,x_2,...,x_n$) una variable aleatoria independiente e idénticamente distribuida, y sea f(x) una función de valor real. La ley de los grandes números establece que:
$$ E(f(x)) ≈ \frac{1}{n} \sum_{i=1}^{n} f(x_i) $$

donde $n$ es el número de de datos o muestras.

**Una primera aproximación**:

Generar distribuciones de lso retornos para los activos o para el portafolio. Basado en la simulación por Monte Carlo de los posibles retornos o parámetros estimados de la distribución, se calcula el VaR.

In [ ]:
# Parametros de la simulacion
#(1-alpha)
n_simulations = 10000   # No. de retornos aleatorios en cada activo
cov = retornos.cov()    # Matriz de covarianzas de los activos

In [ ]:
# Muestras correlacionadas usando Cholesky
np.random.seed(123)
L = np.linalg.cholesky(cov)
Z = np.random.normal(size=(n_simulations, n))
retornos_sim = np.dot(Z,L.T) + mu.values
#retornos_sim

In [ ]:
# Retornos simulados del portafolio
ret_port_sim = np.dot(retornos_sim, w)

In [ ]:
# VaR del portafolio
var_port_pct = np.percentile(ret_port_sim, 100*(1-alpha))      # VaR en porcentaje
var_port_val = - var_port_pct * inversion                       # VaR en unidades monetarias

print("VaR Histórico del activo: ")
print(f"Activo:   {-var_port_pct:.2%}   →  ${var_port_val:,.0f}")

In [ ]:
# Valor del portafolio al día siguiente
valor_port_sim = inversion*np.exp(ret_port_sim)
#perdidas_sim = inversion - valor_port_sim

In [ ]:
plt.figure(figsize=(4,3))
plt.hist(ret_port_sim, bins=30,density=True)
plt.axvline(x=-var_port_pct, color='red', linestyle='--', label=f'VaR 99%: {-var_port_pct:.2%}')
plt.title('Distribución de Rendimientos')
plt.xlabel('Rendimiento diario')
plt.ylabel('Densidad')
plt.legend()
plt.grid()

**¿Qué otras alternativas tienen para generar muestras aleatorias correlacioandas?
Es decir, de qué otra forma se pueden generar esos aletorios correlacionados sin usar Cholesky.**



---
---


**Otra forma común**:

Usando trayectorias de las posibles realizaciones de los precios de las acciones o del valor del portafolio. Basado en la simulación por Monte Carlo de los posibles precios del activo o portafolio, es un VaR histórico pero con valores simulados. Por lo general se asume que el precio del activo sigue un movimiento Browniano geométrico (MBG), de forma que:


$$ S_t = S_{t-1} e^{\big(\mu - 1/2 \sigma^2\big)\Delta t + \sigma \sqrt(\Delta t) Z}\  \text{con}\ \ Z \sim N(0,1)  $$



---
---



## Pros y contras del VaR por MC

**Ventajas**
No requiere supuestos distributivos
Funciona bien con estructuras no lineales

**Desventajas**
Requiere una muestra grande

El análisis paramétrico del VaR sugiere pérdidas potenciales para cada acción con un nivel de confianza del 95 %. El VaR histórico, derivado del rendimiento pasado, indica niveles de riesgo variables entre las acciones. La simulación de Monte Carlo proporciona más información sobre las pérdidas potenciales en diferentes escenarios simulados.
Estos métodos, en conjunto, permiten a los inversionistas y analistas financieros cuantificar y prepararse para el riesgo potencial de caída de sus carteras, mejorando así la toma de decisiones en las estrategias de gestión de riesgos.

---
---

### Limitaciones del VaR

1. No es subaditivo (no siempre respeta diversificación).

2. Ignora las pérdidas más allá del cuantil $𝛼$.

3. Depende fuertemente de los supuestos de distribución.

**Subaditividad**

La subaditividad es una propiedad de las medidas de riesgo que establece que el riesgo de una portafolio combinada no debe ser mayor que la suma de los riesgos de sus componentes individuales; la idea es que la diversificación debería reducir el riesgo. Sin embargo, el Valor en Riesgo (VaR) generalmente no cumple esta propiedad, ya que puede resultar en un riesgo mayor para el portafolio combinada que la suma de los riesgos individuales, lo que significa que el VaR no es una medida de riesgo coherente.

*Representación formal*:

Si $M$ es un espacio convexo, es decir, si $L_1 \in M$ y $L_2 \in M$ implica que $L_1 + L_2 \in M$. Para tod $L_1, L_2 \in M$ tenemos que $\lambda(L1 + L2) ≤ \lambda(L1) + \lambda(L2)$. La idea principal de este propiedad es que “una combinación no crea riesgo adicional”, esto refleja la idea de que el riesgo se puede reducir diversificando los activos. Ver Artzner et al. (1999), para más detalles.

**Hull, Ch 11. Coherent risk measure (Properties)**

1. Monotonicity: If a portfolio produces a worse result than another portfolio for every state of the world, its risk measure should be greater.
2. Translation Invariance: If an amount of cash K is added to a portfolio, its risk measure should go down by K.
3. Homogeneity: Changing the size of a portfolio by a factor λ while keeping the relative amounts of different items in the portfolio the same should result in the risk measure being multiplied by λ.
4. **Subadditivity**: *The risk measure for two portfolios after they have been merged should be no greater than the sum of their risk measures before they were merged.*

## Valor en Riesgo Condicional (CVaR) o Expected Shortfall (ES)


CVaR mide la pérdida esperada dado que la pérdida excedió el VaR.

Definición:

$$ CVaR_{\alpha}(L) = E ( L | L \geq VaR_{\alpha} (L)) $$

Representa la pérdida promedio en el peor $(1−𝛼)%$ de los casos.

**Ventajas:**

* Es mayor que el VaR.

* Captura mejor el riesgo de cola.

* Es coherente con la teoría de medidas de riesgo (Adoptado en Basilea II/III como estándar para el capital regulatorio de riesgo de mercado).

In [ ]:
# VaR (en retornos)
alpha = 0.99
z_alpha = norm.ppf(1-alpha)
VaR_normal = mu_port + sigma_port * z_alpha
print(f"VaR al {alpha:.1%}: {VaR_normal:.4%}")

In [ ]:
# CVaR / ES parametrico (distribución Normal)
ES_normal = mu_port - sigma_port * norm.pdf(z_alpha) / (1 - alpha)
print(f"ES al {alpha:.1%}: {ES_normal:.4%}\n")

In [ ]:
# Otra forma de estimar el ES:
# ES_normal2 = (1-alpha)**-1 * norm.pdf(norm.ppf(1-alpha))* sigma_port - mu_port
# print(f"ES  {alpha:.1%}: {ES_normal2:.4%}\n")

In [ ]:
# CVaR / ES histórico
VaR_hist = np.quantile(ret_port, 1 - alpha)
ES_hist = ret_port[ret_port<= VaR_hist].mean()
print(f"VaR al {alpha:.1%}: {ES_hist:.4%}")

**Conclusiones**

**VaR**: herramienta estándar, pero limitada en eventos extremos.

**CVaR**: captura pérdidas más allá del VaR, preferida en regulación (Basilea II/III).

Ambos deben usarse en conjunto con otras métricas (stress testing, escenarios, etc.).

## Stress Testing

Supongamos un shock adverso del -5%/-20% en todas las acciones.

In [ ]:
# Definir shock por activo (en retornos): aquí -10% cada uno
exposure = inversion * w                                # exposición monetaria por activo
valor_port_actual = valor_port[-1]                      # valor al último día
stress_shock = np.array([-0.15, -0.10, -0.05, -0.20])
#stress_shock = np.repeat(-0.10, n)                      # Escenario alternativo del shock

# Calcular pérdidas monetarias por activo
pnl_stress_by_asset = stress_shock * exposure

# Pérdida total del portafolio
pnl_stress_port = pnl_stress_by_asset.sum()
valor_post_stress = valor_port_actual + pnl_stress_port

In [ ]:
print("\n=== Escenario de estrés aplicado ===")
for col, expos, shock, pnl in zip(acciones, exposure, stress_shock, pnl_stress_by_asset):
    print(f"{col:8s}: Exposición={expos:,.2f}  Shock={shock:+.2%}  ->  P&L={pnl:,.2f}")

print(f"\nValor portafolio antes del stress: {valor_port_actual:,.2f}")
print(f"Pérdida total portafolio (estrés): {pnl_stress_port:,.2f}")
print(f"Valor portafolio post-estrés:      {valor_post_stress:,.2f}")
print(f"VaR histórico (99%):               {var_port_val:,.2f}")


## Otras distribuciones

In [ ]:
# Distribución t-Studet
from scipy.stats import norm, t
tdf, tloc, tscale = t.fit(ret_port)                 # loc (media) | scale (desviacion)
#pdf_t = t.pdf(x, df=tdf, loc=tloc, scale=tscale)          # 	Probability density function - Calcula densidad (pdf)
#tloc, tscale   # Parameters

In [ ]:
# Si tdf es bajo (ej. 3 o 5), tiene colas más pesadas, por lo que es útil para retornos financieros.

In [ ]:
# Calculo del VaR
h = 10  # dias para escalar
k = tdf                    # k grados de libertad (df degree of freedom) - (ret_port)-1
q_est = t.ppf(1-alpha, k)  # Percentil/cuantil
pdf_std = t.pdf(q_est, k)  # PDF estándar en q_est

VaR_t = q_est * tscale - tloc                               # q:est = t.ppf(alpha, k)
print(f"VaR (t-Student) al 99% (with k-df): {VaR_t:.4%}")

In [ ]:
# Otra forma:
#print(t.ppf(alpha, k, loc=tloc, scale=tscale))

Comparación:

In [ ]:
print(f"VaR 99% Normal is: {var_port_pct:.4%}")

In [ ]:
# Calculo del ES
ES_t = (1-alpha)**-1 * ((k + q_est**2)/(k- 1)) * t.pdf(q_est, k)* tscale - tloc # sigma_port - mu_port
print(f"ES (t-Student) al 99% (with k-df): {-ES_t:.4%}")

In [ ]:
print(f"ES 99% Normal is: {ES_normal:.4%}")

---

In [ ]:
## End ...